# Spark SQL Examples

Run the code cells below. This is the same code from the previous screencast.

In [2]:
import findspark

In [4]:
# path to spark file
findspark.init("/usr/local/Cellar/apache-spark/3.0.1/libexec")

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
# create session
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL") \
    .getOrCreate()

In [7]:
# read json
path = "sparkify_log_small.json"
user_log = spark.read.json(path)

In [8]:
# show first row
user_log.take(1)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

In [9]:
# show column schema
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Create a View And Run Queries

The code below creates a temporary view against which you can run SQL queries.

In [10]:
# creates temporary view
user_log.createOrReplaceTempView("user_log_table")

In [11]:
# show first two rows using view
spark.sql("SELECT * FROM user_log_table LIMIT 2").show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

In [12]:
# multi line syntax
spark.sql('''
          SELECT * 
          FROM user_log_table 
          LIMIT 2
          '''
          ).show()

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
|   Lily Allen|Logged In|Elizabeth|     F|            7|   Chase|195.23873| free|Shreveport-Bossie...|   PUT|NextSong|1512718541284|     5027|      

In [13]:
# find number of rows
spark.sql('''
          SELECT COUNT(*) 
          FROM user_log_table 
          '''
          ).show()

+--------+
|count(1)|
+--------+
|   10000|
+--------+



In [14]:
# find info for userId 1046
spark.sql('''
          SELECT userID, firstname, page, song
          FROM user_log_table 
          WHERE userID == '1046'
          '''
          ).collect()

[Row(userID='1046', firstname='Kenneth', page='NextSong', song='Christmas Tears Will Fall'),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='Be Wary Of A Woman'),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='Public Enemy No.1'),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='Reign Of The Tyrants'),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='Father And Son'),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='No. 5'),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='Seventeen'),
 Row(userID='1046', firstname='Kenneth', page='Home', song=None),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='War on war'),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='Killermont Street'),
 Row(userID='1046', firstname='Kenneth', page='NextSong', song='Black & Blue'),
 Row(userID='1046', firstname='Kenneth', page='Logout', song=None),
 Row(userID='1046', firstname='Kenneth'

In [15]:
# show distinct values for the page column
spark.sql('''
          SELECT DISTINCT page
          FROM user_log_table 
          ORDER BY page ASC
          '''
          ).show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



# User Defined Functions

In [16]:
# create a function: get_hour that changes datetime to hour
spark.udf.register("get_hour", lambda x: int(datetime.datetime.fromtimestamp(x / 1000.0).hour))

<function __main__.<lambda>(x)>

In [17]:
# run function in sql query
spark.sql('''
          SELECT *, get_hour(ts) AS hour
          FROM user_log_table 
          LIMIT 1
          '''
          ).collect()

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046', hour='17')]

In [18]:
# show how many songs were played in the hour
songs_in_hour = spark.sql('''
          SELECT get_hour(ts) AS hour, COUNT(*) as plays_per_hour
          FROM user_log_table
          WHERE page = "NextSong"
          GROUP BY hour
          ORDER BY cast(hour as int) ASC
          '''
          )

In [19]:
songs_in_hour.show()

+----+--------------+
|hour|plays_per_hour|
+----+--------------+
|   0|           276|
|   1|           348|
|   2|           358|
|   3|           375|
|   4|           249|
|   5|           216|
|   6|           228|
|   7|           251|
|   8|           339|
|   9|           462|
|  10|           479|
|  11|           484|
|  12|           430|
|  13|           362|
|  14|           295|
|  15|           257|
|  16|           248|
|  17|           369|
|  18|           375|
|  19|           456|
+----+--------------+
only showing top 20 rows



# Converting Results to Pandas

In [20]:
# convert to pandas
songs_in_hour_pd = songs_in_hour.toPandas()

In [21]:
print(songs_in_hour_pd)

   hour  plays_per_hour
0     0             276
1     1             348
2     2             358
3     3             375
4     4             249
5     5             216
6     6             228
7     7             251
8     8             339
9     9             462
10   10             479
11   11             484
12   12             430
13   13             362
14   14             295
15   15             257
16   16             248
17   17             369
18   18             375
19   19             456
20   20             454
21   21             382
22   22             302
23   23             352


# 5 Quiz Section

## Q1: Which page did user id ""(empty string) NOT visit?

In [82]:
# original answer
spark.sql('''
          SELECT DISTINCT(page)
          FROM user_log_table 
          WHERE userID == ''
          '''
          ).collect()

[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]

In [86]:
# we first find where userId is ''
# we join this to a table with all the avialable options in page
# only want the pages where user did not go to: will be null when right joined with full table
spark.sql('''SELECT *
    FROM
    (SELECT DISTINCT page
    FROM user_log_table
    WHERE userId = '') AS user_pages
    RIGHT JOIN (
        SELECT DISTINCT page
        FROM user_log_table) AS all_pages
    ON user_pages.page = all_pages.page
    WHERE user_pages.page IS NULL;''').show()

+----+----------------+
|page|            page|
+----+----------------+
|null|Submit Downgrade|
|null|       Downgrade|
|null|          Logout|
|null|   Save Settings|
|null|        Settings|
|null|        NextSong|
|null|         Upgrade|
|null|           Error|
|null|  Submit Upgrade|
+----+----------------+



## Q2: Why might you prefer to use SQL over data frames? Why might you prefer dataframes over SQL?

SQL is often used by data analysts and data scientists. Dataframes can be presented better.

## Q3: How many female users do we have in the data set?

In [43]:
# group by gender, count distinct userId
spark.sql('''
          SELECT gender, COUNT(DISTINCT userId)
          FROM user_log_table 
          GROUP BY gender
          HAVING gender == 'F';
          '''
          ).collect()

[Row(gender='F', count(DISTINCT userId)=462)]

## Q4: How many songs were played from the most played artist?

In [48]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [53]:
spark.sql("""
    SELECT artist, count(song) song_counts
    FROM user_log_table
    GROUP BY artist
    ORDER BY song_counts DESC
    LIMIT 5;
    """).show()

+--------------------+-----------+
|              artist|song_counts|
+--------------------+-----------+
|            Coldplay|         83|
|       Kings Of Leon|         69|
|Florence + The Ma...|         52|
|            BjÃÂ¶rk|         46|
|       Dwight Yoakam|         45|
+--------------------+-----------+



## Q5: How many songs do users listen to on average between visiting our home page?

In [54]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [68]:
spark.sql("""
    SELECT sessionId, AVG(count(song))
    FROM user_log_table
    WHERE page == 'Home'
    GROUP BY sessionId, count(song)
    LIMIT 10;
    """).show()

AnalysisException: aggregate functions are not allowed in GROUP BY, but found count(user_log_table.`song`);;
GlobalLimit 10
+- LocalLimit 10
   +- Aggregate [sessionId#19L, count(song#20)], [sessionId#19L, avg(count(song#20)) AS avg(count(song))#700]
      +- Filter (page#17 = Home)
         +- SubqueryAlias user_log_table
            +- Relation[artist#7,auth#8,firstName#9,gender#10,itemInSession#11L,lastName#12,length#13,level#14,location#15,method#16,page#17,registration#18L,sessionId#19L,song#20,status#21L,ts#22L,userAgent#23,userId#24] json
